In [9]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.tools import tool

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
model = ChatOpenAI(model="gpt-4o-mini",
                   temperature=0.1,
                   max_tokens=500)

model.invoke("Latentview analytics").content

"LatentView Analytics is a data analytics and insights firm that specializes in providing advanced analytics, data science, and AI-driven solutions to businesses. Founded in 2006 and headquartered in Chennai, India, the company focuses on helping organizations leverage data to drive decision-making, improve operational efficiency, and enhance customer experiences.\n\n### Key Services Offered by LatentView Analytics:\n\n1. **Data Analytics**: They provide services that include descriptive, predictive, and prescriptive analytics to help businesses understand their data and make informed decisions.\n\n2. **Data Engineering**: This involves building robust data pipelines and architectures to ensure that data is collected, stored, and processed efficiently.\n\n3. **Machine Learning and AI**: LatentView develops machine learning models and AI solutions tailored to specific business needs, enabling automation and advanced insights.\n\n4. **Business Intelligence**: They offer tools and dashboa

In [6]:
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent

In [7]:
# class Agentstate(TypedDict):
    

In [8]:
def add_numbers(state):
    result = state["num1"] + state["num2"]
    print(f"The addition result is {result}")
    return Command(goto="multiply",update={"sum": result})

In [11]:
@tool
def transfer_to_multiplication_expert():
    """Ask multiplation agent for help"""
    return
    
@tool
def transfer_to_addition_expert():
    """Ask addition agent for help"""
    return

In [12]:
llm_with_tools = model.bind_tools([transfer_to_multiplication_expert, transfer_to_addition_expert])

In [14]:
response = llm_with_tools.invoke("What is 2+2")

In [15]:
response.tool_calls

[{'name': 'transfer_to_addition_expert',
  'args': {},
  'id': 'call_JUSpQq2wUdUHUwRjTXYrq5sI',
  'type': 'tool_call'}]

In [94]:
from typing_extensions import Literal
from langgraph.graph import MessagesState, StateGraph, START, END

In [117]:
def additional_expert(state:MessagesState)-> Command[Literal["multiplication_expert", "__end__"]]:
    
    system_prompt = (
        "You are an addition expert, you can ask the multiplication expert for help with multiplication."
        "Always do your portion of calculation before the handoff."
    )
    
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    
    ai_msg = model.bind_tools([transfer_to_multiplication_expert]).invoke(messages)
    
    
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        
        return Command(
            goto="multiplication_expert", update={"messages": [ai_msg, tool_msg]}
        )
    return {"messages": [ai_msg]}

In [ ]:
##Agent2
def multiplication_expert(state:MessagesState)-> Command[Literal["additional_expert", "__end__"]]:
    
    system_prompt = (
        "You are a multiplication expert, you can ask an addition expert for help with addition. "
        "Always do your portion of calculation before the handoff."
    )
    
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    ai_msg = model.bind_tools([transfer_to_addition_expert]).invoke(messages)
    
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(goto="additional_expert", update={"messages": [ai_msg, tool_msg]})
    return {"messages": [ai_msg]}

In [119]:
graph = StateGraph(MessagesState)

In [120]:
graph.add_node("additional_expert",additional_expert)
graph.add_node("multiplication_expert",multiplication_expert)

In [121]:

graph.add_edge(START, "additional_expert")

In [122]:
app = graph.compile()

In [123]:
app.invoke({"messages":[("user","what's (3 + 5) * 12. Provide me the output")]})

NameError: name 'llm' is not defined

In [110]:
from langchain_community.tools import DuckDuckGoSearchRun